In [1]:
#此练习，我们将使用逻辑回归来识别手写数字（0到9）。 我们将扩展我们在练习2中写的逻辑回归的实现，
#并将其应用于一对一的分类。 让我们开始加载数据集。 它是在MATLAB的本机格式，所以要加载它在Python，我们需要使用一个SciPy工具。
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
data = loadmat('ex3data1.mat')#加载数据
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
data['X'].shape, data['y'].shape#y是标签，x是特征

((5000, 400), (5000, 1))

In [4]:
#已经加载了数据。图像在martix X中表示为400维向量（其中有5,000个）。 400维“特征”是原始20 x 20图像中每个像素的灰度强度。
#类标签在向量y中作为表示图像中数字的数字类。
#第一个任务是将逻辑回归实现修改为完全向量化（即没有“for”循环）。这是因为向量化代码除了简洁外，还能够利用线性代数优化。
#并且通常比迭代代码快得多。但是，如果从练习2中看到我们的代价函数已经完全向量化实现了，所以我们可以在这里重复使用相同的实现。

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))#可以通过np.matrix()函数将一个变量转换为numpy型矩阵

In [9]:
def cost(theta, X, y, learningRate):
    # INPUT：参数值theta，数据X，标签y，学习率
    # OUTPUT：当前参数值下的交叉熵损失
    # TODO：根据参数和输入的数据计算交叉熵损失函数
    
    # STEP1：将theta, X, y转换为numpy类型的矩阵
    theta =np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # STEP2：根据公式计算损失函数（不含正则化）
    cross_cost =np.multiply(-y, np.log(sigmoid(X * theta.T)))-np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
   
    # STEP3：根据公式计算损失函数中的正则化部分
    reg =  (learningRate / (2 * len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))#正则项
   
    # STEP4：把上两步当中的结果加起来得到整体损失函数
    whole_cost=np.sum(cross_cost)/len(X)+reg#len（X）相当于样本个数，做一个平均
    
    return whole_cost

In [10]:
#向量化的梯度函数，.T代表转置
def gradient(theta, X, y, learningRate):
    # INPUT：参数值theta，数据X，标签y，学习率
    # OUTPUT：当前参数值下的梯度
    # TODO：根据参数和输入的数据计算梯度
    
    # STEP1：将theta, X, y转换为numpy类型的矩阵
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # STEP2：将theta矩阵拉直（转换为一个向量）
    parameters =int(theta.ravel().shape[1])#theta数量
    
    # STEP3：计算预测的误差   
    error = sigmoid(X * theta.T) - y
    
    # STEP4：根据上面的公式计算梯度
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # STEP5：由于j=0时不需要正则化，所以这里重置一下
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

In [11]:
#现在已经定义了代价函数和梯度函数，是构建分类器的时候了。 有10个可能的类，并且由于逻辑回归只能一次在2个类之间进行分类，需要多类分类的策略。
#任务是实现一对一全分类方法，其中具有k个不同类的标签就有k个分类器，每个分类器在“类别 i”和“不是 i”之间决定。 
#将把分类器训练包含在一个函数中，该函数计算10个分类器中的每个分类器的最终权重，并将权重返回为k X（n + 1）数组，其中n是参数数量。
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta

In [12]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [13]:
np.unique(data['y'])#看下有几类标签

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [14]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-2.38280215e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30424698e-03, -7.17025128e-10,  0.00000000e+00],
       [-3.18403182e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.45919625e-03, -5.08397686e-04,  0.00000000e+00],
       [-4.79873344e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.87102343e-05, -2.48777984e-07,  0.00000000e+00],
       ...,
       [-7.98425629e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.94038795e-05,  7.20955711e-06,  0.00000000e+00],
       [-4.57331493e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33639371e-03,  9.99383654e-05,  0.00000000e+00],
       [-5.40434738e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16605966e-04,  7.88388836e-06,  0.00000000e+00]])

In [15]:
#现在准备好最后一步 - 使用训练完毕的分类器预测每个图像的标签。 
#对于这一步，将计算每个类的类概率，对于每个训练样本（使用当然的向量化代码），并将输出类标签为具有最高概率的类。
#可以使用np.argmax()函数找到矩阵中指定维度的最大值
def predict_all(X, all_theta):
    # INPUT：参数值theta，测试数据X
    # OUTPUT：预测值
    # TODO：对测试数据进行预测
    
    # STEP1：获取矩阵的维度信息
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # STEP2：把矩阵X加入一行零元素
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # STEP3：把矩阵X和all_theta转换为numpy型矩阵
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # STEP4：计算样本属于每一类的概率
    h = sigmoid(X * all_theta.T)
    
    # STEP5：找到每个样本中预测概率最大的值
    h_argmax = np.argmax(h, axis=1)
    
    # STEP6：因为我们的数组是零索引的，所以我们需要为真正的标签+1
    h_argmax = h_argmax + 1
    
    return h_argmax

In [16]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.46%
